In [1]:
import numpy as np
import pandas as pd
import re

pd.options.display.max_columns = 100
idx = pd.IndexSlice

%matplotlib inline
import matplotlib.pyplot as pt
import seaborn as sb
sb.set_style('whitegrid')

import wikifunctions as wf

import requests, json, re, time, os, pprint

path = 'catlists\\'
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import unquote, quote
from copy import deepcopy

In [2]:
#set up an empty cat-list pair legend
#df=pd.DataFrame(columns=['category','list','suffixes'])
#df.head()
#df.to_csv(path+'keyPairs.csv',index=False)

In [78]:
def get_list(listName):
    l=listName.replace(' ','_')
    #df=pd.DataFrame()
    r = requests.get('https://en.wikipedia.org/wiki/'+str(l))
    con = r.content
    #print(con)
    df = pd.read_html(con)
    if len(df)>1:
        if len(df[0])==1:
            df=df[1:]
        d=pd.DataFrame()
        for i in range(1,len(df)):
            d=d.append(df[i])
        df=d
    else:
        df=df[0]
    #df=df[0]
    #df.to_csv(path+l+'.csv')
    df['inList']=listName
    df['listLink']='https://en.wikipedia.org/wiki/'+str(l)
    return df

In [54]:
c=get_list('List of LGBT politicians in the United States')
c.tail()

,vteLGBT Politicians and Office-holders by country,vteLGBT Politicians and Office-holders by country.1,inList,listLink
0,Australia Netherlands United Kingdom United St...,Australia Netherlands United Kingdom United St...,List of LGBT politicians in the United States,https://en.wikipedia.org/wiki/List_of_LGBT_pol...


In [4]:
def standardizeLists(df):
    #name
    dz=pd.DataFrame(columns=['name'])
    try:
        dz['name']=df['Name']
    except:
        try:
            dz['name']=df['Judge']
        except:
            print(df.head())
            return  
    #birth and/or death years
    try:
        dz['LGBTQ Identity']=df['Sexual orientation and/or gender identity']
    except:
        try:
            dz['LGBTQ Identity']=df['Sexual orientation and/or gender identity[2]']
        except:
            try:
                dz['LGBTQ Identity']=df['Orientation(s) & identification(s)']
            except:
                try:
                    dz['LGBTQ Identity']=df['Sexual orientation/gender identity']
                except:
                    pass
    try:
        dz['years']=df['Dates']
    except:
        try:
            dz['years']=df['Birth year']
        except:
            try:
                dz['years']=df['Lifetime']
            except:
                try:
                    dz['years']=df['Lifetime[1]']
                except:
                    pass
    try:
        dz['years'].apply(lambda s:s.strip('b.'))
    except:
        pass
    #nationality
    try:
        dz['country']=df['Nationality']
    except:
        try:
            dz['country']=df['Country']
        except:
            pass
    #pronouns maybe
    try:
        dz['pronouns']=df['Pronouns']
    except:
        pass
    #misc info
    try:
        dz['misc']=df['Comments']
    except:
        try:
            dz['misc']=df['Notable as']
        except:
            try:
                dz['misc']=df['Notes']
            except:
                pass
    dz['inList']=df['inList']
    dz['listLink']=df['listLink']
    
    return dz

In [5]:
def get_catMembers(categoryName, d=3):
    c=categoryName.replace(' ','_')
    catMembers = wf.get_category_members('Category:'+c,
                                        depth=d)
    
    df=pd.DataFrame()
    df['name']=catMembers
    df['wikiLink']=df['name'].apply(lambda s: 'https://en.wikipedia.org/wiki/'+str(s.replace(',','').replace(' ','_')))
    df['inCategory']=categoryName
    return df

In [5]:
#d=get_list('List of transgender people')
#d=standardizeLists(d)
#d.head()

In [6]:
#this method was the last thing causing problems so I just pulled it apart in a loop later

def diffListCat(categoryName,listName,suffixes=[]):
    a=get_catMembers(categoryName)
    b=get_list(listName,suffixes)
    b=standardizeLists(b)
    try:
        c=list(set(b['name']) and set(a['name']))
        print(c)
    except:
        return
    notInList=b[b['name'] not in c]
    notInCategory=a[a['name'] not in c]
    notInList['addToList']=listName
    notInCategory['addToCategory']=categoryName
    return [notInList, notInCategory]

In [13]:
df=pd.read_csv(path+'keyPairs.csv',keep_default_na=False,index_col=False)
addToList=pd.DataFrame()
addToCat=pd.DataFrame()
for index, row in df.iterrows():
    try:
        ls,ct=diffListCat(categoryName=row[0],listName=row[1],suffixes=row[2])
        addToList.append(ls)
        addToCat.append(ct)
    except:
        pass

In [ ]:
addToList.to_csv('addToList.csv',index=False)
addToCat.to_csv('addToCat.csv',index=False)

In [33]:
#testing the get_list() method
ls=get_list('List of LGBT writers')
ls.head()

,Name,Lifetime,Nationality,Genre,Notable works,References,inList,listLink
0,Caio Fernando Abreu,1948–1996,Brazilian,"short stories, plays","Morangos Mofados, dragões não conhecem o paraíso",[1],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
1,Barry D. Adam,b. 1952,Canadian,scholar,"The Rise of a Gay and Lesbian Movement, The Gl...",[2],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
2,Evan Adams,b. 1966,Canadian,playwright,"Dreams of Sheep, Snapshots, Dirty Dog River an...",[3],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
3,Nathan Niigan Noodin Adler,b. ?,Canadian,horror,"Wrist, Ghost Lake",[4],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
4,Etel Adnan,b. 1925,Lebanese-American,poet,Sea and Fog,[5],List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [35]:
#testing the standardizeLists() method
standardized_ls=standardizeLists(ls)
standardized_ls.head()

,name,years,country,inList,listLink
0,Caio Fernando Abreu,1948–1996,Brazilian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
1,Barry D. Adam,b. 1952,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
2,Evan Adams,b. 1966,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
3,Nathan Niigan Noodin Adler,b. ?,Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
4,Etel Adnan,b. 1925,Lebanese-American,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [36]:
cm=get_catMembers('LGBT writers')
cm.head()

,name,wikiLink,inCategory
0,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,LGBT writers
1,Rob Astbury,https://en.wikipedia.org/wiki/Rob_Astbury,LGBT writers
2,Anton Enus,https://en.wikipedia.org/wiki/Anton_Enus,LGBT writers
3,Narelda Jacobs,https://en.wikipedia.org/wiki/Narelda_Jacobs,LGBT writers
4,Patricia Karvelas,https://en.wikipedia.org/wiki/Patricia_Karvelas,LGBT writers


In [42]:
z=standardized_ls.merge(cm, on='name', how='inner').drop_duplicates()

In [47]:
notInCategory=standardized_ls[~standardized_ls.name.isin(cm['name'].unique())]
notInCategory.head()

,name,years,country,inList,listLink
6,Magaly Alabau,b. 1945,Cuban American,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
15,Lisa Alther,b. 1944,American,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
21,Patrick Anderson,1915–1979,English/Canadian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
22,Albalucía Angel,b. 1939,Colombian,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...
23,Núria Añó,b. 1973,Spanish,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...


In [48]:
notInList=cm[~cm.name.isin(standardized_ls['name'].unique())]
notInList.head()

,name,wikiLink,inCategory
0,List of LGBT writers,https://en.wikipedia.org/wiki/List_of_LGBT_wri...,LGBT writers
1,Rob Astbury,https://en.wikipedia.org/wiki/Rob_Astbury,LGBT writers
2,Anton Enus,https://en.wikipedia.org/wiki/Anton_Enus,LGBT writers
3,Narelda Jacobs,https://en.wikipedia.org/wiki/Narelda_Jacobs,LGBT writers
4,Patricia Karvelas,https://en.wikipedia.org/wiki/Patricia_Karvelas,LGBT writers


In [89]:
df=pd.read_csv(path+'keyPairs.csv',keep_default_na=False,index_col=False)
#addToList=pd.DataFrame()
#addToCat=pd.DataFrame()
addToList=pd.read_csv(path+'addToList.csv',keep_default_na=False)
addToCat=pd.read_csv(path+'addToCat.csv',keep_default_na=False)

In [90]:
df.head()

,category,list,suffixes
0,LGBT sportspeople,List of LGBT sportspeople,
1,LGBT African Americans,List of LGBT African Americans,
2,LGBT writers,List of LGBT writers,
3,LGBT YouTubers,List of LGBT YouTubers,
4,LGBT heads of government,List of openly LGBT heads of government,


In [91]:
df=df[4:5].append(df[6:7])
df.head()

,category,list,suffixes
4,LGBT heads of government,List of openly LGBT heads of government,
6,LGBT judges,List of LGBT jurists in the United States,


In [92]:
for index, row in df.iterrows():
    listName=row['list']
    categoryName=row['category']
    ls=get_list(listName)
    #ls.head()
    standardized_ls=standardizeLists(ls)
    #standardized_ls.head()
    cm=get_catMembers(categoryName)
    #cm.head()
    notInCategory=standardized_ls[~standardized_ls.name.isin(cm['name'].unique())]
    notInCategory['addThisCategoryTag']=categoryName
    #notInCategory.head()
    notInList=cm[~cm.name.isin(standardized_ls['name'].unique())]
    notInList['addToThisList']=listName
    notInList['listLink']='https://en.wikipedia.org/wiki/'+str(listName.replace(' ','_'))
    #notInList.head()
    addToList=pd.concat([addToList,notInList],ignore_index=True)
    addToCat=pd.concat([addToCat,notInCategory],ignore_index=True)


<ipython-input-92-e6a6aed31f29>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notInCategory['addThisCategoryTag']=categoryName
<ipython-input-92-e6a6aed31f29>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notInList['addToThisList']=listName
<ipython-input-92-e6a6aed31f29>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

                 Judge     Court/Agency & Years of Service (if known)  \
0     Charles Adams[1]     Santa Clara County Superior Court (2018– )   
1        Joel Agron[2]  San Bernardino County Superior Court (2017– )   
2  Ronald E. Albers[3]   San Francisco County Superior Court (2009– )   
3     Daniel Anders[4]    Philadelphia Court of Common Pleas (2007– )   
4  Carolyn Archbold[5]         29th District Court (resigned in 2003)   

  State/Territory    Status                                     inList  \
0      California    active  List of LGBT jurists in the United States   
1      California    active  List of LGBT jurists in the United States   
2      California    active  List of LGBT jurists in the United States   
3    Pennsylvania    active  List of LGBT jurists in the United States   
4        Michigan  deceased  List of LGBT jurists in the United States   

                                            listLink  
0  https://en.wikipedia.org/wiki/List_of_LGBT_jur...  
1  htt

AttributeError: 'NoneType' object has no attribute 'name'

In [80]:
addToList.tail()

,name,wikiLink,inCategory,addToThisList,listLink
17630,Jóhanna Sigurðardóttir,https://en.wikipedia.org/wiki/Jóhanna_Sigurðar...,LGBT heads of government,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...
17631,Leo Varadkar,https://en.wikipedia.org/wiki/Leo_Varadkar,LGBT heads of government,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...
17632,William II of the Netherlands,https://en.wikipedia.org/wiki/William_II_of_th...,LGBT heads of government,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...
17633,Richard Hatfield,https://en.wikipedia.org/wiki/Richard_Hatfield,LGBT heads of government,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...
17634,Jerónimo Saavedra,https://en.wikipedia.org/wiki/Jerónimo_Saavedra,LGBT heads of government,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...


In [81]:
addToCat.tail()

,name,LGBTQ Identity,years,country,inList,listLink,addThisCategoryTag,misc
222,Francesco Italia,NaN,NaN,Italy,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...,LGBT heads of government,NaN
223,Lori Lightfoot,NaN,NaN,United States,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...,LGBT heads of government,NaN
224,Claudia López Hernández,NaN,NaN,Colombia,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...,LGBT heads of government,NaN
225,Todd Gloria,NaN,NaN,United States,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...,LGBT heads of government,NaN
226,Christopher Pappas,NaN,NaN,South Africa,List of openly LGBT heads of government,https://en.wikipedia.org/wiki/List_of_openly_L...,LGBT heads of government,NaN


In [71]:
addToList.head()

,name,wikiLink,inCategory,addToThisList,listLink
0,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...,LGBT sportspeople,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...
1,LGBT athletes in the Olympic and Paralympic Games,https://en.wikipedia.org/wiki/LGBT_athletes_in...,LGBT sportspeople,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...
2,Cecilia Carranza,https://en.wikipedia.org/wiki/Cecilia_Carranza,LGBT sportspeople,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...
3,Yamila Rodríguez,https://en.wikipedia.org/wiki/Yamila_Rodríguez,LGBT sportspeople,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...
4,Sebastián Vega (basketball),https://en.wikipedia.org/wiki/Sebastián_Vega_(...,LGBT sportspeople,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...


In [72]:
addToCat.head()

,name,LGBTQ Identity,years,country,inList,listLink,addThisCategoryTag,misc
0,Brian Anderson,Gay[13],b. 1976,United States,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...,LGBT sportspeople,
1,Eric Anderson,Gay[14],b. 1968,United States,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...,LGBT sportspeople,
2,Jen Armbruster,Lesbian[18],b. 1975,United States,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...,LGBT sportspeople,
3,Zeb Atlas,Bisexual[24][25],b. 1970,United States,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...,LGBT sportspeople,
4,Jason Ball,Gay[31],b. 1988,Australia,List of LGBT sportspeople,https://en.wikipedia.org/wiki/List_of_LGBT_spo...,LGBT sportspeople,


In [20]:
addToList.to_csv(path+'addToList.csv',index=False)
addToCat.to_csv(path+'addToCat.csv',index=False)